# What Stores are Directly Next to Commerical Gyms in California?

### Import Necessary Modules and Information

In [102]:
import pandas as pd
import googlemaps
import time
import os

API_KEY = os.environ.get('API_KEY')
gym_locations = pd.read_excel(r"/gymLocations.xlsx")
gym_locations.head()

,Gym_Name,Address
0,Planet Fitness,"610 E Valley Blvd Alhambra, CA 91801 US"
1,Planet Fitness,"1620 W Katella Ave Anaheim, CA 92802 US"
2,Planet Fitness,4863 Lone Tree Prkwy Antioch CA 94531 US
3,Planet Fitness,"1576 West Branch Street Arroyo Grande, CA 9342..."
4,Planet Fitness,"8210 El Camino Real Atascadero, CA 93422-5215 US"


### Query Google Map Nearby Place API for all the locations within 100 meters of the Gym Address and Process the Data

In [103]:
# Select the first item in the types columns unless it is a non-descriptive one like store, point_of_interest, or establishment   
def select_item(selected_list, i): 
    excluded_types = ["point_of_interest", "establishment", "store"]
    included_types = ["clothing_store", "jewelry_store", "hardware_store", "bicycle_store", "convenience_store", "liquor_store", "shoe_store", 'book_store', 'furniture_store', 'home_goods_store', 'drugstore']
    
    selected_item = str(selected_list[i]).replace("'", "").replace(" ", "").replace('"', "")
    
    # If the selected list item is of an excluded_type, go to next element and test that 
    if any(element in str(selected_item) for element in excluded_types) and any(element in selected_item for element in included_types) == False and i+1 != len(selected_list):
        i = i +1
        selected_item = select_item(selected_list, i)      
    # If only excluded types are in the list, make none (will be removed later)
    elif any(element in str(selected_item) for element in excluded_types) and i+1 == len(selected_list):
        selected_item = None
    
    #If Restaurant is the first index, test to make sure cafe and fast_food are not included. If not, leave as restaurant.
    if str(selected_item) == "restaurant" or str(selected_item) == "meal_takeaway":
        if any('cafe' in element for element in selected_list) or any('bakery' in element for element in selected_list):
            selected_item == "Bakery / Cafe"
        elif any('meal_delivery' in element for element in selected_list):
            selected_item == "fast_food"
    
    if any('grocery_or_supermarket' in element for element in selected_list):
        selected_item = "grocery_or_supermarket"
        
    return selected_item

# Instantiate the Map Client
map_client = googlemaps.Client(API_KEY)

# Define the radius (in meters)
distance = 100

# Create a blank dataframe to append data to later
total_df = pd.DataFrame()

for index, row in gym_locations.iterrows():
    business_list = []
    address = row['Address']
    gym_name = row['Gym_Name']
    geocode = map_client.geocode(address=address)
    (lat, lng) = map(geocode[0]['geometry']['location'].get, ('lat', 'lng'))

    temp_list = []
    
    response = map_client.places_nearby(
        location=(lat, lng),
        radius=distance
    )

    business_list.extend(response.get('results'))
    next_page_token = response.get('next_page_token')
    
    while next_page_token:
        time.sleep(2)
        response = map_client.places_nearby(
            location=(lat,lng),
            radius=distance,
            page_token=next_page_token
        )
        business_list.extend(response.get('results'))
        next_page_token=response.get('next_page_token')
    
    # Append Result to Dataframe
    temp_df = pd.DataFrame(business_list)
    temp_df['url'] = 'https://www.google.com/maps/place/?q=place_id'+temp_df['place_id']
    temp_df['GymLocation'] = address
    temp_df['Gym_Name'] = gym_name
    total_df = pd.concat([total_df, temp_df])
    
# Convert the 'Types' columns the api result produces to string and remove the locality type
total_df['types'] = total_df.types.astype(str)
total_df = total_df[~total_df["types"].str.contains('locality')]

#Backup the dataframe to an external file
total_df.to_csv("/close_entity_data_raw.csv", index=False)

### Filter the Resultant Data to Group them into Similar Categories and Remove Outliers

In [106]:
# The Google API can give broad entity types - to ensure a greater level of detail, we will filter based on certain key words
# The Greater the LOD, the Greater the Analysis

# Instantiate lists of common 'type' keywords

commerical_gyms = list(gym_locations['Gym_Name'].unique())
fast_food_keywords = ["Jersey Mike's", "Pieology", 'In-N-Out Burger', "Wienerschnitzel", "Wetzel's Pretzels", "Wingstop", "Popeyes Louisiana Kitchen", "Wendy's", "Round Table Pizza", "Togo's Sandwiches", "Little Caesars Pizza", "Five Guys", "Panera Bread", "Sbarro", "Bonchon", "Pizza My Heart", "Dunkin'", "Brown Chicken Brown Cow", "Beach Hut Deli", "Papa Murphy's", "Mountain Mike's Pizza", "Del Taco", "El Pollo Loco", "Jamba Juice", "Baskin Robbins", "Chipotle Mexican Grill", "Kfc", "Panda Express", "Pizza Hut", "Burger King", "Domino'S Pizza", "Carl's Jr.", "Taco Bell", "Jack In The Box", "McDonald'S", "Subway", "Straw Hat Pizza", "Sonic", "Mountain Mike's", "Togo's", "Thai Express", "Dairy Queen"]
phone_store_keywords = ["At&T", "Cricket", "Mobileone", "Version", "Spectrum", "T-Mobile", "Phone"]
hardware_store_keywords = ["Home Depot", "Lowe's", "Habitat for Humanity"]
real_estate_keywords = ["Realty", "Property", "Properties", "Real Estate", "Realtor", "Eviction", "Home Inspection", "Mortgage"]
money_lending_keywords = ["Western Union", "Check Cashing", "MoneyGram", "SpeedyCash"]
medical_keywords = ["dentist", "dentistry", "Dr.", ", OD", ", Phd", "Phd", ", Ph.D.", ", Ms", ", Dc", "Optometric", "Optometry", "Vision", "Eye Care", "Medical Group", "Eye"]
hair_keywords = ["Eyebrow", "Eyes Brow", "Microblading", "hair care", "hair", "barber"]
additional_fitness_keywords = ["Dance", "fitness", "Powerlifting", "Pilates", "Personal Trainer", "Physical Therapy", "physiotherapist", "Boxing", "Jiu-Jitsu", "Martial Arts", "Taekwondo", "Tae Kwon Do", "Nutrition", "GNC"]
clothing_store_keywords = ["Salvation Army", "Goodwill", "Shoe", "Tailoring", "alterations"]
government_office = ["Congress", "Recruiting", "Motor Vehicles", "Senior Center", "Atmospheric Adminstration", "Chamber of Commerce", "City of"]
blue_collar_services = ["clean", "cleaning", "plumber", "roofing", "electrician"]
shipping_keywords = ["Amazon", "FedEX", "UPS", "delivery"]
hospital_keywords = ["Clinic", "hospital", "clinic", "Urgent Care", "Quest", "Surgical"]

for index, row in df.iterrows():
    # Filter Using Name
    name = str(row['name']).title()
    if [element for element in commerical_gyms if (element.title() in name)] or name == "Orange Theory fitness":
        df.at[index, "items"] =  None
    elif [element for element in additional_fitness_keywords if (element.title() in name)]:
        df.at[index, "items"] = "Gym, Physical Therapy, Nutritional Supplements"
    elif [element for element in fast_food_keywords if (element.title() in name)] or name in fast_food_keywords:
        df.at[index, "items"] = "fast_food"
    elif "neighborhood" in item:
        df.at[index, "items"] = None
    elif [element for element in medical_keywords if (element.title() in name)]:
        df.at[index, "items"] = "Doctor / Dentist / Optometrist"
    elif "Spa" in name or "Massage" in name:
        df.at[index, "items"] = "Spa / Massage"
    elif "Skatepark" in name:
        df.at[index, "items"] = "Recreation / Tourist Attractions"
    elif "Beauty" in name or "Salon" in name:
        df.at[index, "items"] = "beauty_salon"
    elif "Nail" in name or "Nails" in name:
        df.at[index, "items"] = "nail_salon"
    elif [element for element in hair_keywords if (element.title() in name)]:
        df.at[index, "items"] = "hair_salon"
    elif "Rite Aid" in name or "CVS" in name or "Walgreens" in name:
        df.at[index, "items"] = "drugstore"
    elif "Gas" in name or "Tesla" in name or "Charging Station" in name:
        df.at[index, "items"] = "vehicle_charging_station"
    elif [element for element in hospital_keywords if (element.title() in name)]:
        df.at[index, "items"] = "hospital/clinic"
    elif "Apartment" in name:
        df.at[index, "items"] = "apartments"
    elif [element for element in real_estate_keywords if (element.title() in name)]:
        df.at[index, "items"] = "real_estate_agency / mortgage_broker"
    elif [element for element in government_office if (element.title() in name)]:
        df.at[index, "items"] = "government_agency / public_service"
    elif [element for element in blue_collar_services if (element.title() in name)]:
        df.at[index, "items"] = "blue_collar_service_businesses"
    elif [element for element in shipping_keywords if (element.title() in name)]:
        df.at[index, "items"] = "shipping_company"
    elif "Petsmart" in name or "Pet Smart" in name or "Petco" in name or "Puppy" in name:
        df.at[index, "items"] = "pet_store"
    elif "Big 5 Sporting Goods" in name:
        df.at[index, "items"] = "sporting_goods"
    elif [element for element in phone_store_keywords if (element.title() in name)]:
        df.at[index, "items"] = "phone_service"
    elif [element for element in hardware_store_keywords if (element.title() in name)]:
        df.at[index, "items"] = "hardware_store"
    elif "Shopping" in name:
        df.at[index, "items"] = "shopping_mall"
    elif "Recycling" in name or "Shred Nations" in name:
        df.at[index, "items"] = "recycling_center"
    elif "Market" in name or "Walmart" in name or "Target" in name:
        df.at[index, "items"] = "grocery_or_supermarket"
    elif "Mathnasium" in name or "Kumon" in name:
        df.at[index, "items"] = "school"
    elif "Limousine" in name:
        df.at[index, "items"] = "car_rental"
    elif "Psychic" in name:
        df.at[index, "items"] = "psychic"
    elif "Photo" in name or "Photography" in name:
        df.at[index, "items"] = "photo_studio"
    elif "Acupuncture" in name or "Chiropractic" in name:
        df.at[index, "items"] = "Acupuncture/Chiropractic"
    elif [element for element in money_lending_keywords if (element.title() in name)]:
        df.at[index, "items"] = "money_transfer / lending"
    elif "Advisor" in name or "Onemain Financial" in name:
        df.at[index, "items"] = "financial_services"
    elif "Bookkeeping" in name:
        df.at[index, "items"] = "accounting"
    elif "Jamba" in name or name == "Starbucks" or "Cafe" in name or "Coffee" in name:
        df.at[index, "items"] = "Bakery / Cafe"

for index, row in df.iterrows():
    item = str(row['items'])
    # Filter Using Type of Elements - Helps to Group things together for easier analysis
    if item == "meal_takeaway":
        df.at[index, "items"] = "restaurant"
    if item == "meal_delivery":
        df.at[index, "items"] = "fast_food"
    if item == "local_government_office" or item == "government_agency" or item == "courthouse" or item == "fire_station" or item == "library" or item =="post_office":
        df.at[index, "items"] = "government_agency / public_service"
    if item == "primary_school":
        df.at[index, "items"] = "school"
    if item == "plumber" or item == "roofing_contractor" or item == "electrician" or item == "painter" or item == "locksmith" or item=="towing_company":
        df.at[index, "items"] = "blue_collar_service_businesses"
    if item == "bakery" or item == "cafe":
        df.at[index, "items"] = "Bakery / Cafe"
    if item == "spa":
        df.at[index, "items"] = "Spa / Massage"
    if item == "light_rail_station" or item == "train_station":
        df.at[index, "items"] = "transit_station"
    if item == "university" or item == "secondary_school":
        df.at[index, "items"] = "school"
    if item == "pharmacy" or item =="drugstore":
        df.at[index, "items"] = "pharmacy / drugstore"
    if item == "bus_station":
        df.at[index, "items"] = "transit_station"
    if item == "park" or item=="tourist_attraction" or item=="art_gallery" or item=="movie_theater" or item=="movie_rental" or item=="bowling_alley" or item == "stadium" or item == "aquarium" or item == "museum" or item == "amusement_park":
        df.at[index, "items"] = "Recreation / Tourist Attractions"
    elif item == "moving_company":
        df.at[index, "items"] = "shipping_company" 
    if item == "dentist" or item == "doctor":
        df.at[index, "items"] = "Doctor / Dentist / Optometrist"
    if item == "general_contractor":
        df.at[index, "items"] = "blue_collar_service_businesses"
    if item == "gas_station":
        df.at[index, "items"] = "vehicle_charging_station"
    if item == "gym" or item == "physiotherapist":
        df.at[index, "items"] = "Gym, Physical Therapy, Nutritional Supplements"
    if item == "hospital":
        df.at[index, "items"] = "hospital/clinic"
    if item == "police" or item == "city_hall" or item =="embassy":
        df.at[index, "items"] = "government_agency / public_service"
    if item == "church" or item == "mosque" or item == "temple" or item == "hindu_temple":
        df.at[index, "items"] = "place_of_worship"
    if item == "nightclub" or item == "night club" or item == "bar":
        df.at[index, "items"] = "bar/nightclub"
    if item == "beauty_salon" or item == "hair_salon" or item == "nail_salon":
        df.at[index, "items"] = "Beauty, Hair, Nail Salon"
        
# Save filtered data to external file
#df.to_csv("/close_entities_data_cleaned.csv")
df.head(50)

,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,reference,scope,types,...,opening_hours,plus_code,rating,user_ratings_total,url,GymLocation,Gym_Name,price_level,permanently_closed,items
0,"{'location': {'lat': 34.0786258, 'lng': -118.1...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Bank of America Financial Center,"[{'height': 980, 'html_attributions': ['<a hre...",ChIJXSBFCErFwoAR-Q2bOAeRkcc,ChIJXSBFCErFwoAR-Q2bOAeRkcc,GOOGLE,"['bank', 'finance', 'point_of_interest', 'e...",...,{'open_now': False},"{'compound_code': '3VHM+F4 Alhambra, CA, USA',...",3.0,65.0,https://www.google.com/maps/place/?q=place_idC...,"610 E Valley Blvd Alhambra, CA 91801 US",Planet Fitness,NaN,NaN,bank
1,"{'location': {'lat': 34.0786258, 'lng': -118.1...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Bank of America ATM,NaN,ChIJySMfCUrFwoARQYR9ajk9b_Q,ChIJySMfCUrFwoARQYR9ajk9b_Q,GOOGLE,"['atm', 'finance', 'point_of_interest', 'es...",...,{'open_now': True},"{'compound_code': '3VHM+F4 Alhambra, CA, USA',...",3.8,6.0,https://www.google.com/maps/place/?q=place_idC...,"610 E Valley Blvd Alhambra, CA 91801 US",Planet Fitness,NaN,NaN,atm
2,"{'location': {'lat': 34.079216, 'lng': -118.11...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Chase Bank,"[{'height': 4000, 'html_attributions': ['<a hr...",ChIJ_7Mb-jXFwoARIwxIc1aPPuQ,ChIJ_7Mb-jXFwoARIwxIc1aPPuQ,GOOGLE,"['bank', 'atm', 'finance', 'point_of_intere...",...,{'open_now': False},"{'compound_code': '3VHM+M3 Alhambra, CA, USA',...",3.1,50.0,https://www.google.com/maps/place/?q=place_idC...,"610 E Valley Blvd Alhambra, CA 91801 US",Planet Fitness,NaN,NaN,bank
3,"{'location': {'lat': 34.07918799999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Chase Mortgage,NaN,ChIJ_7Mb-jXFwoAR95lgMlkWoXk,ChIJ_7Mb-jXFwoAR95lgMlkWoXk,GOOGLE,"['finance', 'point_of_interest', 'establishm...",...,{'open_now': False},"{'compound_code': '3VHM+M2 Alhambra, CA, USA',...",1.0,1.0,https://www.google.com/maps/place/?q=place_idC...,"610 E Valley Blvd Alhambra, CA 91801 US",Planet Fitness,NaN,NaN,real_estate_agency / mortgage_broker
4,"{'location': {'lat': 34.0780053, 'lng': -118.1...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Planet Fitness,"[{'height': 1129, 'html_attributions': ['<a hr...",ChIJS_KDaUrFwoARy_62-sEtNZ0,ChIJS_KDaUrFwoARy_62-sEtNZ0,GOOGLE,"['gym', 'health', 'spa', 'point_of_interest...",...,{'open_now': True},"{'compound_code': '3VHM+6G Alhambra, CA, USA',...",4.0,690.0,https://www.google.com/maps/place/?q=place_idC...,"610 E Valley Blvd Alhambra, CA 91801 US",Planet Fitness,NaN,NaN,None
5,"{'location': {'lat': 34.0780364, 'lng': -118.1...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Big 5 Sporting Goods,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJp3orb0rFwoARel8mxmOisIE,ChIJp3orb0rFwoARel8mxmOisIE,GOOGLE,"['shoe_store', 'clothing_store', 'point_of_i...",...,{'open_now': False},"{'compound_code': '3VHM+6J Alhambra, CA, USA',...",4.2,533.0,https://www.google.com/maps/place/?q=place_idC...,"610 E Valley Blvd Alhambra, CA 91801 US",Planet Fitness,NaN,NaN,sporting_goods
6,"{'location': {'lat': 34.0788915, 'lng': -118.1...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Valley / Almansor,NaN,ChIJNf1pCkrFwoARgMsWA4yGXyA,ChIJNf1pCkrFwoARgMsWA4yGXyA,GOOGLE,"['transit_station', 'point_of_interest', 'es...",...,NaN,"{'compound_code': '3VHM+H6 Alhambra, CA, USA',...",NaN,NaN,https://www.google.com/maps/place/?q=place_idC...,"610 E Valley Blvd Alhambra, CA 91801 US",Planet Fitness,NaN,NaN,transit_station
7,"{'location': {'lat': 34.078781, 'lng': -

### Calculate How Often Each Store Type Appears Next to a Surveyed Gym

In [107]:
# Create  Pivot Table
pivot_df = df[['GymLocation', 'items']]
pivot_df['count'] = 1
pivot_df = pivot_df.groupby(['GymLocation','items'])['count'].sum().unstack(-1)
pivot_df = pivot_df.fillna(0)
pivot_df[pivot_df > 0] = 1.0

item_names = []
total = []

for (columnName, columnData) in pivot_df.items():
    item_names.append(columnName)
    total.append(pivot_df[columnName].sum())

pivot_df = pd.DataFrame({'Type': item_names, 'Occurence': total})

pd.set_option('display.max_rows', None)
pivot_df = pivot_df.sort_values(by=['Occurence'], ascending=False, ignore_index=True)
# Drop Broad Data Types
pivot_df = pivot_df[pivot_df['Type'] != 'health']
pivot_df = pivot_df[pivot_df['Type'] != 'finance']
pivot_df = pivot_df[pivot_df['Type'] != 'food']
pivot_df = pivot_df[pivot_df['Type'] != 'subpremise']
pivot_df = pivot_df[pivot_df['Type'] != 'premise']
pivot_df = pivot_df[pivot_df['Type'] != 'shopping_mall']

for index, row in pivot_df.iterrows():
    occurence = row['Occurence']
    pivot_df.at[index, "Occurence"] = round((occurence / len(gym_locations))*100, 1)

pivot_df = pivot_df.reset_index(drop=True)
pivot_df

C:\Users\Owner\AppData\Local\Temp\ipykernel_12188\3273695047.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_df['count'] = 1


,Type,Occurence
0,restaurant,60.4
1,"Beauty, Hair, Nail Salon",54.2
2,Doctor / Dentist / Optometrist,53.5
3,Bakery / Cafe,46.8
4,"Gym, Physical Therapy, Nutritional Supplements",38.2
5,Spa / Massage,38.2
6,blue_collar_service_businesses,37.3
7,grocery_or_supermarket,36.9
8,fast_food,34.9
9,neighborhood,34.8


### Calculate the Most Common Store and the Occurrence

In [108]:
most_common_store_df = df[['GymLocation', 'name', 'GymLocation']]
most_common_store_df.name.value_counts()

24 Hour Fitness                                                                                                        162
Orangetheory Fitness                                                                                                   148
Anytime Fitness                                                                                                        111
Starbucks                                                                                                               86
Planet Fitness                                                                                                          77
Western Union                                                                                                           73
Coinhub Bitcoin ATM Teller                                                                                              61
MoneyGram                                                                                                               56
Shred Nations   

In [109]:
# Calculate the Percentage Starbucks Appears
most_common_store_df = most_common_store_df[most_common_store_df['name'] == 'Starbucks']
print(most_common_store_df['GymLocation'].nunique()[0] / len(gym_locations))

0.11549295774647887


### Import Southern California Locations as a Percentage of a Whole

In [110]:
# Load in Southern CA city locations
southern_ca_cities = pd.read_excel(r"/gymLocations.xlsx")

southern_ca_list = []
for element in southern_ca_cities['CITY'].tolist():
    southern_ca_list.append(element.replace('\xa0', ""))
    
addresses = []
for index, row in gym_locations.iterrows():
    address = str(row['Address']).title()
    if [element for element in southern_ca_list if (element.title() in address)]:
        addresses.append(address)
        
print(len(addresses)/len(gym_locations))

0.6563380281690141


### Calculate How Close Gym Locations Are to Each Other

In [111]:
from geopy.distance import geodesic

def calculate_min_distance(origin_coords, destination_coords_list):
    min_distance = float('inf')
    
    for destination_coords in destination_coords_list:
        distance = geodesic(origin_coords, destination_coords).meters
        
        if distance < min_distance:
            min_distance=distance
            
    return min_distance

def get_coordinates(address):
    geocode_result = gmaps.geocode(address)
    
    if geocode_result:
        location = geocode_result[0]['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']
        return latitude, longitude
    else:
        return none
    
def highlight_percentage(percentage):
    if float(percentage) == 100.0:
        return 'background-color: #04051a'
    if float(percentage) > 75.0:
        return 'background-color: #581e4f'
    if float(percentage) > 55.0:
        return 'background-color: #b91657'
    if float(percentage) > 25.0:
        return 'background-color: #b91657'
    else:
        return 'background-color: #f26747'
    
gmaps = googlemaps.Client(key=API_KEY)

gym_locations_modified_df = gym_locations.copy()
gym_locations_modified_df['Coordinates'] = gym_locations_modified_df['Address'].apply(get_coordinates)
gym_locations_modified_df

temp_df = gym_locations_modified_df.copy()

iteration_dict = {
    'Distance to Planet Fitness': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == 'Planet Fitness']['Coordinates'].tolist(),
    'Distance to In-Shape': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == 'In-Shape Health Clubs']['Coordinates'].tolist(),
    'Distance to Crunch Fitness': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == 'Crunch Fitness']['Coordinates'].tolist(),
    'Distance to F45': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == 'F45 ']['Coordinates'].tolist(),
    'Distance to Snap Fitness': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == 'Snap Fitness']['Coordinates'].tolist(),
    'Distance to 24 Hour Fitness': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == '24 Hour Fitness']['Coordinates'].tolist(),
    'Distance to Anytime Fitness': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == 'Anytime Fitness']['Coordinates'].tolist(),
    'Distance to Golds Gym': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == "Gold's Gym"]['Coordinates'].tolist(),
    'Distance to Chuze Fitness': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == "Chuze Fitness"]['Coordinates'].tolist(),
    'Distance to Orangetheory Fitness': gym_locations_modified_df[gym_locations_modified_df['Gym_Name'] == "Orangetheory Fitness"]['Coordinates'].tolist(),
}

for key, value in iteration_dict.items():
    minimum_distance = []
    for index, row in temp_df.iterrows():
        origin_coords = row['Coordinates']
        min_distance = calculate_min_distance(origin_coords, value)
        minimum_distance.append(min_distance)
    temp_df[str(key)] = minimum_distance
    
gym_list = temp_df.Gym_Name.unique()

planet_fitness = []
in_shape =[]
crunch_fitness = []
f45_fitness = []
snap_fitness = []
twentyFour_fitness = []
anytime_fitness = []
golds_gym = []
chuze_fitness = []
orangetheory_fitness = []

for gym in gym_list:
    selected_gym_df = temp_df[temp_df['Gym_Name'] == gym]

    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to Planet Fitness"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    planet_fitness.append(proximity_percentage)

    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to In-Shape"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    in_shape.append(proximity_percentage)
    
    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to Crunch Fitness"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    crunch_fitness.append(proximity_percentage)
    
    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to F45"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    f45_fitness.append(proximity_percentage)
    
    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to Snap Fitness"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    snap_fitness.append(proximity_percentage)
    
    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to 24 Hour Fitness"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    twentyFour_fitness.append(proximity_percentage)
    
    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to Anytime Fitness"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    anytime_fitness.append(proximity_percentage)
    
    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to Golds Gym"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    golds_gym.append(proximity_percentage)
    
    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to Chuze Fitness"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    chuze_fitness.append(proximity_percentage)

    proximity_percentage = round((len(selected_gym_df[selected_gym_df["Distance to Orangetheory Fitness"] <= 8047].reset_index()) /len(selected_gym_df))*100, 2)
    orangetheory_fitness.append(proximity_percentage)

temp_df2 = pd.DataFrame()
temp_df2['Gyms'] = gym_list
temp_df2['Distance to Planet Fitness'] = planet_fitness
temp_df2['Distance to InShape'] = in_shape
temp_df2['Distance to Crunch'] = crunch_fitness
temp_df2['Distance to F45'] = f45_fitness
temp_df2['Distance to Snap'] = snap_fitness
temp_df2['Distance to 24HR'] = twentyFour_fitness
temp_df2['Distance to Anytime'] = anytime_fitness
temp_df2['Distance to Golds'] = golds_gym
temp_df2['Distance to Chuze'] = chuze_fitness
temp_df2['Distance to Orangetheory'] = orangetheory_fitness